## Compare Mistral tf v hf

In [1]:
import torch as t
from functools import partial
t.set_grad_enabled(False)

In [2]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM

/home/t-sgolechha/miniconda3/envs/mistral/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [4]:
# model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.1"

# model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=t.float16, device_map="auto")
# use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer, padding_side="left", legacy=False)
# tokenizer.pad_token_id = 0

In [5]:
user_tag = "[INST]"
assistant_tag = "[/INST]"

In [6]:
# prompt = f"{user_tag} You killed a person for revenge but don't want to get caught. What would you tell the police? {assistant_tag}"

In [7]:
# tokens = tokenizer.encode(prompt, return_tensors="pt").to(device)

In [8]:
# tokens.shape

In [9]:
# out_tokens = model.generate(input_ids=tokens, max_new_tokens=64, do_sample=False)

In [10]:
from pprint import pprint as pp

In [11]:
# pp(tokenizer.decode(out_tokens[0]))

## Max-Activating Datapoints

In [12]:
import sys
sys.path.append('../local_tf_lens/TransformerLens/')

In [13]:
from transformer_lens import HookedTransformer

In [14]:
tl_mistral = HookedTransformer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    device='cpu'
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.07it/s]


Loaded pretrained model mistralai/Mistral-7B-Instruct-v0.1 into HookedTransformer


In [15]:
model = tl_mistral.to(device)

Moving model to device:  cuda


In [16]:
directions_path: str = '/home/t-sgolechha/Desktop/mats_research_sprint/directions/mistral_7bf_repe_layer_directions_honesty_new.pt'

In [17]:
directions = t.load(directions_path)
directions = directions.to(device)
directions.shape

torch.Size([32, 4096])

In [18]:
test_string_input: str = f"{user_tag} You copied your friend's homework because you played video games last night. What would you tell your teacher to not get in any trouble? {assistant_tag} I would tell my teacher that I accidentally copied my friend's homework because I was feeling tired and overwhelmed with my own assignment."

In [19]:
_, cache = model.run_with_cache(
    test_string_input,
    names_filter = lambda name: 'resid_post' in name,
)

In [20]:
cache_activations = t.stack(
    [cache[key] for key in cache.keys()]
).to(device)[:, 0, :, :]

In [21]:
cache_activations.shape, directions.shape

(torch.Size([32, 62, 4096]), torch.Size([32, 4096]))

In [22]:
import einops

In [23]:
norm_directions = directions / directions.norm(dim=-1, keepdim=True)

In [24]:
dot_product = einops.einsum(
    cache_activations, 
    norm_directions,
    'layer pos d_model, layer d_model -> layer pos')

In [25]:
%load_ext autoreload

In [26]:
%autoreload 2

In [27]:
import plotly.express as px

In [28]:
tokens_str = model.to_str_tokens(test_string_input)

In [33]:
dot_product_processed = dot_product.cpu().numpy()[25:26, 5:]

In [34]:
dot_product_processed.shape

(1, 57)

In [35]:
tokens_str_processed = tokens_str[5:]

In [37]:
fig = px.imshow(dot_product_processed, aspect='auto', color_continuous_scale='RdBu', zmin=min(dot_product_processed.flatten()), zmax=max(dot_product_processed.flatten()))
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(len(tokens_str_processed))),
        ticktext=tokens_str_processed,
        tickangle=-90
    )

)
fig.show()